In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
import multiprocessing as mp
import time
from os.path  import basename
import requests
from PIL import Image
from IPython.display import display
import pandas as pd

In [8]:
def get_info_morizon(url):
    
    driver = webdriver.Chrome()
    driver.get(url)
    driver.maximize_window()
    
    
    try:
        nazwa = driver.find_element(By.XPATH, '//*[@id="app"]/div[2]/main/div[4]/section/div[4]/h1').text
    except:
        nazwa=None

    try:
        cena = driver.find_element(By.XPATH, '//*[@id="basic-info-price-row"]/div/span[1]').text
    except:
        cena=None

    try:
        pokoje = driver.find_element(By.XPATH, '//*[@id="app"]/div[2]/main/div[4]/section/div[2]/span[1]/span').text.split(" ")[0]
    except:
        pokoje = None
    
    try:
        lokalizacja = driver.find_element(By.XPATH, '//*[@id="app"]/div[2]/main/div[4]/section/div[3]/h2/span[2]').text
    except:
        lokalizacja = None


    try:
        x=''
        dane = driver.find_elements(By.XPATH, '//*[@id="app"]/div[2]/main/div[4]/section/div[6]')
        for i in dane:
            x = x + i.text + ', '
        try:
            dane = driver.find_elements(By.XPATH, '//*[@id="app"]/div[2]/main/div[4]/section/div[7]')
            for i in dane:
                x = x + i.text + ', '
        except:
            pass
        wynik = obrabianie(x)
    except:
        wynik=None

    
    try:
        miejsce_parkingowe = driver.find_element(By.XPATH,'//*[@id="app"]/div[2]/main/div[4]/section/div[10]/ul/li[3]/span' ).text
    except:
        miejsce_parkingowe = None


    try:
        button = driver.find_element(By.XPATH, '//*[@id="app"]/div[2]/main/div[4]/section/div[4]/button')
        button.click()
    except:
        pass

    try:
        opis = driver.find_element(By.XPATH, '//*[@id="app"]/div[2]/main/div[4]/section/div[4]/div').text
    except:
        opis = None
  
    col = ['nazwa_ogloszenia', 'link','cena', 'opis', 'lokalizacja','pokoje', 'miejsce parkingowe']
    dane = [[nazwa,url,cena, opis, lokalizacja,pokoje, miejsce_parkingowe]]
    czesc = pd.DataFrame(data=dane,columns=col)
    koncowy = pd.concat([czesc, wynik], ignore_index = False ,axis=1)
    driver.quit()

    

    return koncowy

In [10]:
get_info_morizon('https://www.morizon.pl/oferta/sprzedaz-mieszkanie-wroclaw-krzyki-gen-ignacego-pradzynskiego-96m2-mzn2043231298')

,nazwa_ogloszenia,link,cena,opis,lokalizacja,pokoje,miejsce parkingowe,powierzchnia,balkon,taras,piętro,rok budowy,stan wykonczenia
0,Okolice ul. Prądzyńskiego /rozkład/do remontu,https://www.morizon.pl/oferta/sprzedaz-mieszka...,1 100 000 zł,Do sprzedania mieszkanie w centrum Wrocławia 3...,"Południe,",3,None,96 m²,None,None,1/5,1930,DO REMONTU KAPIT.


In [9]:
get_info_morizon('https://www.morizon.pl/oferta/sprzedaz-mieszkanie-wroclaw-krzyki-gen-romualda-traugutta-34m2-mzn2043233555')

,nazwa_ogloszenia,link,cena,opis,lokalizacja,pokoje,miejsce parkingowe,powierzchnia,balkon,taras,piętro,rok budowy,stan wykonczenia
0,Okolice ul. Traugutta /Inwestycja,https://www.morizon.pl/oferta/sprzedaz-mieszka...,430 000 zł,Na sprzedaż mieszkanie 1-pokojowe o powierzchn...,"Przedmieście Oławskie,",1,None,34 m²,None,None,3/9,2002,WYSOKI STANDARD


In [3]:
def get_links(url):
    links = []
    driver = webdriver.Chrome()
    driver.get(url)
    driver.maximize_window()
    linki = driver.find_elements(By.CSS_SELECTOR, 'a')
    for link in linki:
        try:
            if 'oferta/sprzedaz' in link.get_attribute("href"):
                links.append(link.get_attribute("href"))
        except:
            pass
    return list(set(links))

In [4]:
def obrabianie(dane):
    features_df = pd.DataFrame(columns=['powierzchnia', 'balkon','taras', 'piętro', 'rok budowy','stan wykonczenia'])
    slownik = {'Pow. całkowita':None, 'Balkon':None, 'Taras':None, 'Piętro':None, 'Rok budowy':None, 'Stan nieruchomości':None}
    dane = dane.split('\n')
    nowe=[]
    for i in dane:
       nowe.extend(i.split(', '))
    for i in range(0,len(nowe)):
        if nowe[i] in slownik.keys():
            slownik[nowe[i]] = nowe[i+1]
    slownik['powierzchnia']=slownik['Pow. całkowita']
    slownik['stan wykonczenia']=slownik['Stan nieruchomości']
    data_dict_lower = {key.lower(): value for key, value in slownik.items()}
    features_df.loc[len(features_df)] = data_dict_lower
    return features_df
        

In [5]:
def ogarniacz_linkow(URL, page_start, page_stop):
    oficjalne_linki=[]
    urls = []                 

    for i in range(page_start, page_stop):
        url = URL + '?page=' + str(i)
        urls.append(url)

    for url in urls:
        oficjalne_linki.append(get_links(url))

    if len(oficjalne_linki) > 1:
        oficjalne_linki = [url for sublist in oficjalne_linki for url in sublist]
    else:
        oficjalne_linki = oficjalne_linki[0]
    return oficjalne_linki

In [6]:
def scrapowanie(URL, page_start, page_stop):
    wynik=pd.DataFrame()
    linki = ogarniacz_linkow(URL, page_start, page_stop)
    ile=1
    naile =len(linki)
    for link in linki:
        print(str(ile) + '/' + str(naile))
        i=get_info_morizon(link)
        wynik = pd.concat([wynik, i], ignore_index = True)
        ile = ile +1

    wynik.to_csv('C:/Studia/modelowanie w pythonie/projekt/morizon_tescik.csv', sep = ',', index = False, encoding = 'utf-8') 
    return wynik

In [41]:
scrapowanie('https://www.morizon.pl/mieszkania/wroclaw/', 1, 2)

1/36
2/36
3/36
4/36
5/36
6/36
7/36
8/36
9/36
10/36
11/36
12/36
13/36
14/36
15/36
16/36
17/36
18/36
19/36
20/36
21/36
22/36
23/36
24/36
25/36
26/36
27/36
28/36
29/36
30/36
31/36
32/36
33/36
34/36
35/36
36/36


,nazwa_ogloszenia,link,cena,opis,lokalizacja,pokoje,stan wykonczenia,miejsce parkingowe,powierzchnia,balkon,taras,piętro,rok budowy
0,Opis nieruchomości,https://www.morizon.pl/oferta/sprzedaz-mieszka...,None,Atmo\nNajważniejsze atuty\n✅ Przemyślane rozkł...,"Śródmieście,",3,None,None,"61,85 m²",None,None,3/6,2023
1,Opis nieruchomości,https://www.morizon.pl/oferta/sprzedaz-mieszka...,None,HB1820 – apartamenty w centrum Wrocławia gotow...,"Śródmieście,",3,None,None,"90,18 m²",None,None,4/7,2022
2,Opis nieruchomości,https://www.morizon.pl/oferta/sprzedaz-mieszka...,None,HB1820 – apartamenty w centrum Wrocławia gotow...,"Śródmieście,",3,None,None,"90,01 m²",None,None,1/7,2022
3,Opis nieruchomości,https://www.morizon.pl/oferta/sprzedaz-mieszka...,None,Apartamenty nad Oławką to nowa inwestycja prem...,"Krzyki,",2,None,None,"38,42 m²",Tak,None,None,2025
4,Opis nieruchomości,https://www.morizon.pl/oferta/sprzedaz-mieszka...,None,Najważniejsze atuty\n✅ Wyjątkowa lokalizacja –...,"Śródmieście,",2,None,None,"30,95 m²",None,None,6/6,2024
5,Okolice ul. Prądzyńskiego /rozkład/do remontu,https://www.morizon.pl/oferta/sprzedaz-mieszka...,1 100 000 zł,Do sprzedania mieszkanie w centrum Wrocławia 3...,"Południe,",3,None,None,96 m²,None,None,1/5,1930
6,Opis nieruchomości,https://www.morizon.pl/oferta/sprzedaz-mieszka...,None,Osiedle Zielna\n\nNajważniejsze atuty\nKameral...,"Psie Pole,",2,None,None,"50,20 m²",Tak,None,None,2023
7,Opis nieruchomości,https://www.morizon.pl/oferta/sprzedaz-mieszka...,None,Atmo\nNajważniejsze atuty\n✅ Przemyślane rozkł...,"Śródmieście,",3,None,None,"73,68 m²",None,None,4/6,2023
8,2 pokoje + kuchnia 52m2 duży balkon Wrocław Ps...,https://www.morizon.pl/oferta/sprzedaz-mieszka...,624 000 zł,"Mieszkanie 52m2 w pełni rozkładowe, piętro 3 z...","Psie Pole,",2,None,None,"52,04 m²",Tak,None,3/4,2010
9,Opis nieruchomości,https://www.morizon.pl/oferta/sprzedaz-mieszka...,None,HB1820 – apartamenty w centrum Wrocławia gotow...,"Śródmieście,",3,None,None,"84,96 m²",None,None,3/7,2022
